<a href="https://colab.research.google.com/github/amelievert/JAD-Up/blob/main/Projet_JAD'Up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet d'application Streamlit

3.7.9 . So i Recommend to uninstall all the previous version and install python 3.7 and try running the command: "pip install streamlit" or "pip install --upgrade streamlit"



### Installation Streamlit

In [ ]:
!pip install streamlit

### Configuration du site

In [ ]:
%%writefile streamlit_app.py 

# ______________________________________________________________________________________________________
# Import des bibliothèques
# ______________________________________________________________________________________________________

import streamlit as st
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# ______________________________________________________________________________________________________
# Configuration du site
# ______________________________________________________________________________________________________

st.set_page_config(page_title="JAD'Up",  layout='wide', page_icon='/content/Agence de Marketing.ico')

st.sidebar.title("Sommaire")
st.sidebar.image('/content/Agence de Marketing.ico')

pages = ["Introduction au jeu de données",
         "Analyse",
         "Preprocessing",
         "Challenge de modèles",
         "Pour aller plus loin"]

page = st.sidebar.radio("Aller vers", pages) 


### Préparation des jeux de données

In [ ]:
%%writefile streamlit_app.py 

# ______________________________________________________________________________________________________
# Import du jeu de données
# ______________________________________________________________________________________________________

df = pd.read_csv('/content/bank.csv', sep = ',')


# ______________________________________________________________________________________________________
# Préparation des jeux de données à utiliser
# ______________________________________________________________________________________________________

df2 = df.copy()

# Creation de tranches d'âges
df2['t_age'] = pd.cut(x = df2['age'], bins = [17, 30, 40, 50, 65, 96], labels = ['18-30', '30-40','40-50', '50-65','65-95'])

# Creation de tranches de solde compte bancaire = balance
df2['t_balance'] = pd.qcut(x=df2["balance"], q=4, labels=[1,2,3,4])

# Creation de tranches de durée de contact = duration
df2['t_duration'] = pd.qcut(df2["duration"], q=4, labels=[1,2,3,4])

# Creation de tranches de durée de contact = duration
df2['t_duration'] = pd.qcut(df2["duration"], q=4, labels=[1,2,3,4])

# Creation de tranches de nombre de contact = campaign > Corrige le problème de valeurs abbérantes et limite à 4 contacts
df2['t_campaign'] = pd.cut(x = df2['campaign'], bins = [0, 1, 2, 3, 99], labels = [1, 2, 3, 4])

# Création d'une catégorie pour contact campagne précédente oui/non
df2['contact_last_campaign'] = np.where(df2['pdays']>=0, 'yes', 'no')

# Création de tranches en fonction du délai écoulé
df2['t_pdays'] = pd.cut(x = df2['pdays'], bins = [-2, 0, 200, 999], labels = ['NON CONTACTE', 'MOINS DE 200J', 'PLUS DE 200J'])

# Creation de tranches de nombre de contact avant la campagne
df2['previous'] = pd.cut(x = df2['previous'], bins = [0, 1, 2, 3, 99], labels = [1, 2, 3, 4])

# Suppression des colonnes dummies"ées"
drop_cols=['age','balance','duration','campaign','pdays','previous']
df2 = df2.drop(drop_cols, axis=1)

# Création de dummies
var=['marital','education','poutcome','contact','t_age','t_balance','t_duration','t_campaign','t_pdays','month']
df2= df2.join(pd.get_dummies(df2[var], prefix=var))
df2 = df2.drop(df2[var], axis=1)

# Transformation en numérique
le = LabelEncoder()
df2['job2']= le.fit_transform(df2['job'])
df2 = df2.drop(['job'], axis=1)

# Remplace yes/no par 1/0
var = ["default", "housing","loan","deposit","contact_last_campaign"]
df2[var] = df2[var].replace(('yes', 'no'), (1, 0))



# Construction du site

### Page 0 - Introduction au jeu de données

In [ ]:
%%writefile streamlit_app.py 

# ______________________________________________________________________________________________________
# 1/ Introduction au jeu de données
# ______________________________________________________________________________________________________

if page==pages[0]: 

  st.title("Description du jeu de données")

  st.write("Ce jeu de données est composé de données personnelles sur des clients d’une banque qui ont été “télémarketés” pour souscrire à un produit que l’on appelle un 'dépôt à terme'.")
  st.write("Lorsqu’un client souscrit à ce produit, il place une quantité d’argent dans un compte spécifique et ne pourra pas toucher ces fonds avant l’expiration du terme.")
  st.write("En échange, le client reçoit des intérêts de la part de la banque à la fin du terme.")
  st.write("Le jeu de données est téléchargeable au lien suivant: ")
  st.write("https://www.kaggle.com/janiobachmann/bank-marketing-dataset")


# ---------- Les chiffres clés -----------

  st.header("Les chiffres clés :")
  col1, col2, col3, col4, col5 = st.columns(5)
  col1.write('')
  col2.metric("Nombre de clients", "11 162")
  col3.metric("Nombre de features", "17")
  col4.metric("Proportion des cibles", "47%")
  col5.write('')

# ---------- les variables  -----------

  st.header("Description des variables :")

  var = pd.DataFrame({"Nom des variables": ["age","job","marital","education","default","balance","housing","loan","contact","day","month","duration","campaign","pdays","previous","poutcome","deposit"],
    "Description": ["Age du client","Profession","Statut marital","Niveau d'études","Défaut de paiement","Solde du compte","Prêt immo","Prêt perso",
    "Type de contact","Dernier jour de contact","Dernier mois de  contact","Durée du contact (secondes)","Nombre de contacts","Nb jours écoulés depuis le dernier contact","Nb de contacts",
    "Résultat de la campagne précédente","Résultat de la campagne en cours"]
    })

  st.write(var)

# ---------- Aperçu -----------

  st.header("Aperçu du jeu de données :")
  st.write(df)

# ---------- Ce qu'il faut comprendre -----------

  st.header("Ce qu'il faut retenir :")
  st.write("On remarque que certaines variables sont la résultante de la campagne en cours : ")
  st.write("* contact")
  st.write("* day")
  st.write("* month")
  st.write("* duration")
  st.write("* campaign")
  st.write("La variable [deposit] est notre variable cible.")
  st.write("47% des clients ont répondu favorablement à la campagne (deposit=yes)")



### Page 1 - Analyse du jeu de données

In [ ]:
%%writefile streamlit_app.py 

# ______________________________________________________________________________________________________
# 2/ Analyse du jeu de données
# ______________________________________________________________________________________________________

if page==pages[1]: 

  st.title("Analyse du jeu de données")

# ---------- Fonction de description -----------

  def describe_df(df):
      """
      Fonction améliorée de description des colonnes, elle permet d'identifier :
      le type de la colonne , le nb de valeur vide (nan), le nb de valeurs uniques, le pourcentage de répartition des valeurs
      INPUT : le dataframe
      OUTPUT : tableau d'analyse
      """
      res = pd.DataFrame(index=["Name","Type", "Nan", "Unique","Min","Max","Values","Pourcentage"])
      for col in df.columns:
          df_col = df[col]
          res[col] = [
              df_col.name,
              df_col.dtype,
              df_col.isnull().sum(),
              len(df_col.unique()),
              df_col.min(),
              df_col.max(),
              df_col.unique(),
              (df_col.value_counts(ascending=False, normalize=True) * 100)
                  .apply(int)
                  .to_json(),
          ]
      return res.T

# ---------- Affichage de la description détaillée -----------

  description = st.expander("Afficher la description des colonnes")
  description.dataframe(describe_df(df).astype(str))


# ---------- Les correlations -----------

  col1, col2 = st.columns(2)

# Matrice de correlation

  le = LabelEncoder()
  df2=df.copy()
  for col in df2.columns:
    df2[col]= le.fit_transform(df2[col])
  
  col1.subheader("Matrice de corrélation")

  fig = plt.figure(figsize=(10,10))
  sns.heatmap(df2.corr(), annot=True, cmap='RdBu_r', center=0)
  col1.pyplot(fig)


# Corrélations directes

  tab1, tab2 = col2.tabs(["📈 Chart", "🗃 Coefficients"])

  tab1.subheader("Graphiques des corrélations directes")
  
  corr=pd.DataFrame(df2.corr()["deposit"])
  corr=corr.sort_values("deposit",ascending=False, key=abs)

  fig = plt.figure(figsize=(10,7))
  fig = px.bar(corr,
                 x="Deposit",
                 y=corr.index,
                 template = 'seaborn')
  tab1.plotly_chart(fig, use_container_width=True) 


  tab2.subheader("Coefficients")

  coef=df2.corr()["deposit"]
  tab2.write(coef)

# ---------- Les observations -----------

  st.subheader("Observations :")

  st.write("Dans l'ordre, les variables les plus corrélées (valeur absolue) avec la target 'déposit' sont :")
  st.write("* duration*** = Durée du contact (en secondes)")
  st.write("* contact*** = Type de contact")
  st.write("* housing = Prêt immo")
  st.write("* previous = Nb contacts au cours de la campagne précédente")
  st.write("* pdays = Nb jours écoulés depuis le dernier contact de la campagne précédente")
  st.write("* balance = Solde compte bancaire")
  st.write("*** : attention , deux variables correspondent à des données non connues à priori (avant lancement de la campagne")


# ---------- Les distributions par type de variables -----------

  st.subheader("Les distributions :")
  col3, col4 = st.columns(2)
  df2 = df.copy()
  numerics = df2.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns
  categoricals= df2.select_dtypes(include=['object','category']).columns


# variables numériques

  col3.subheader("Variables numériques")
  tab3, tab4 = col3.tabs(["🗃 Describe", "📈 Chart"])

  describe= df2[numerics].describe().transpose()
  tab3.write(describe)

  option = tab4.selectbox("Choix une variable numérique :",numerics)
  hist = px.histogram(df2,x=option,color="deposit",barmode="group")
  tab4.plotly_chart(hist)

  if option=="age":
    tab4.write("Les âges extrêmes semblent avoir une plus forte adhérence avec la campagne.")
  elif option=="balance":
    tab4.write("RAS")
  elif option=="day":
    tab4.write("RAS")
  elif option=="duration":
    tab4.write("On remarque que plus la durée de contact augmente et plus les clients semblent souscrire à la campagne.")
  elif option=="campaign":
    tab4.write("RAS")
  elif option=="pdays":
    tab4.write("RAS")
  elif option=="previous":
    tab4.write("RAS")

  col3.header("Observations")
  col3.write("On remarque que 8 324 clients n'ont pas été contactés lors de la campagne précédente.")
  col3.write("Lorsque PREVIOUS = 0 alors PDAYS = -1")

# variables catégorielles

  col4.subheader("Variables catégorielles")
  tab5, tab6 = col3.tabs(["🗃 Describe", "📈 Chart"])

  describe= df2[categoricals].describe().transpose()
  tab5.write(describe)

  option = tab6.selectbox("Choix une variable :", categoricals)
  hist = px.histogram(df2,x=option,color="deposit")
  tab6.plotly_chart(hist)


### Page 2 - Préprocessing

In [ ]:
%%writefile streamlit_app.py 

# ______________________________________________________________________________________________________
# 3/ Préprocessing
# ______________________________________________________________________________________________________


if page==pages[2]: 

  st.title("Préprocessing - Modèles prédictifs")

# ---------- Remise à situation d'origine -----------

  # Réimport du fichier
  df2 = df.copy()

# ---------- Temps de preprocessing -----------

  st.write("Attendre le traitement des variables ...")

  import time
  my_bar = st.progress(0)
  for percent_complete in range(100):
      time.sleep(0.1)
      my_bar.progress(percent_complete + 1)


# ---------- Le préprocessing, ça sert à quoi -----------

  expander1 = st.expander("Le préprocessing, ça sert à quoi ?")
  expander1.dataframe(describe_df(df).astype(str))

  expander1.write("Le préprocessing est une de composante essentielle de la data science.")
  expander1.write("Cette étape décrit toutes les transformations effectuées sur le jeu de données initial et indispensables à la création du modèle d'apprentissage fiable et robuste.")
  expander1.write("Les algorithmes d'apprentissage automatique fonctionnent mieux lorsque les données sont présentées dans un format qui met en évidence les aspects pertinents requis pour résoudre un problème.")
  expander1.write("Les fonctions de préprocessing consistent à : ")
  expander1.write("* la transformation des données,")
  expander1.write("* la réduction des données,")
  expander1.write("* la sélection des variables")
  expander1.write("* et à la mise à l'échelle")
  expander1.write("pour restructurer les données brutes sous une forme adaptée à des types particuliers d'algorithmes.")

  from PIL import Image
  image = Image.open('preprocessing.JPG')
  expander1.image(image, caption='Les étapes de préprocessing')

# ---------- Les étapes de préprocessing -----------

  st.header("Les étapes de préprocessing appliquées :")

  code = ''' 
    # Creation de tranches d'âges
    df2['t_age'] = pd.cut(x = df2['age'], bins = [17, 30, 40, 50, 65, 96], labels = ['18-30', '30-40','40-50', '50-65','65-95'])

    # Creation de tranches de solde compte bancaire = balance
    df2['t_balance'] = pd.qcut(x=df2["balance"], q=4, labels=[1,2,3,4])

    # Creation de tranches de durée de contact = duration
    df2['t_duration'] = pd.qcut(df2["duration"], q=4, labels=[1,2,3,4])

    # Creation de tranches de durée de contact = duration
    df2['t_duration'] = pd.qcut(df2["duration"], q=4, labels=[1,2,3,4])

    # Creation de tranches de nombre de contact = campaign > Corrige le problème de valeurs abbérantes et limite à 4 contacts
    df2['t_campaign'] = pd.cut(x = df2['campaign'], bins = [0, 1, 2, 3, 99], labels = [1, 2, 3, 4])

    # Création d'une catégorie pour contact campagne précédente oui/non
    df2['contact_last_campaign'] = np.where(df2['pdays']>=0, 'yes', 'no')

    # Création de tranches en fonction du délai écoulé
    df2['t_pdays'] = pd.cut(x = df2['pdays'], bins = [-2, 0, 200, 999], labels = ['NON CONTACTE', 'MOINS DE 200J', 'PLUS DE 200J'])

    # Creation de tranches de nombre de contact avant la campagne
    df2['previous'] = pd.cut(x = df2['previous'], bins = [0, 1, 2, 3, 99], labels = [1, 2, 3, 4])

    # Suppression des colonnes dummies"ées"
    drop_cols=['age','balance','duration','campaign','pdays','previous']
    df2 = df2.drop(drop_cols, axis=1)

    # Création de dummies
    var=['marital','education','poutcome','contact','t_age','t_balance','t_duration','t_campaign','t_pdays','month']
    df2= df2.join(pd.get_dummies(df2[var], prefix=var))
    df2 = df2.drop(df2[var], axis=1)

    # Transformation en numérique
    le = LabelEncoder()
    df2['job2']= le.fit_transform(df2['job'])
    df2 = df2.drop(['job'], axis=1)

    # Remplace yes/no par 1/0
    var = ["default", "housing","loan","deposit","contact_last_campaign"]
    df2[var] = df2[var].replace(('yes', 'no'), (1, 0))
    '''
  st.code(code, language='python')


# ---------- Arbre de correlations après preprocessing -----------

  st.header("Arbre de correlations après preprocessing :")

  fig = plt.figure(figsize=(20,15))
  plt.title(label="Correlation des features avec la variable cible deposit")
  df2.corr()['deposit'].sort_values().drop('deposit').plot(kind='barh', cmap='RdBu_r')
  st.pyplot(fig)


# ---------- Les enseignements -----------

  st.header("Les enseignements :")

  st.write("On voit clairement que la feature [duration] impacte positivement la campagne dès lors que la valeur est élevée (temps de contact).")
  st.write("Egalement, les clients ayant répondu favorablement à la campagne précédente [poutcome] semblent être les plus susceptibles de renouveler leur action.")
  st.write("Les mois de mars et octobre [month] semblent être les meilleurs mois pour optimiser les leads.")



### Page 3 - Challenge de modèles

In [ ]:
%%writefile streamlit_app.py 

# ______________________________________________________________________________________________________
# 4/ Challenge de modèles
# ______________________________________________________________________________________________________


if page==pages[3]: 

  st.title("Modèles prédictifs")
     

# ---------- Initialisation des biblothèques utilisées -----------

  from sklearn.metrics import accuracy_score, plot_confusion_matrix, roc_curve, roc_auc_score, auc, precision_score, recall_score, classification_report
  from sklearn import linear_model, neighbors, svm, tree, ensemble
  from sklearn.model_selection import GridSearchCV, train_test_split
  import matplotlib.pyplot as plt

  df3=df2.copy()


# ---------- Split jeu entrainement et jeu de test -----------

  # Isoler les features de la target
  target = df3['deposit']
  feats = df3.drop(['deposit'], axis=1)

  # Séparation des données en jeu d'entraînement et de test
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.25)

  # Normaliser les données - MinMaxScaler
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Sauvegarde des résulats de chacun des modèles
  models=[]
  scores =[]
  precision=[]
  rappel=[]
  roc=[]

# ---------- Les 3 modèles -----------

  col1, col2, col3, col4 = st.columns(4)

# Régression logistique -----------------------------------------------------------------------

  with col1:
    expander = st.expander("Modèle RLC")
    rlc = linear_model.LogisticRegression(C=10)
    rlc.fit(X_train, y_train)
        
    expander.metric("Score train", "{:.2%}".format(rlc.score(X_train, y_train)))
    expander.metric("Score test", "{:.2%}".format(rlc.score(X_test, y_test)))
    expander.metric("Precision Score", "{:.2%}".format(precision_score(y_test, rlc.predict(X_test))))

    y_pred = rlc.predict(X_test)
    expander.write("Matrice de confusion :")
    expander.write(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

    # Sauvegarde des résultats
    models.append("Regression logistique")
    scores.append(rlc.score(X_test, y_test))
    precision.append(precision_score(y_test, rlc.predict(X_test)))
    rappel.append(recall_score(y_test, rlc.predict(X_test)))
    roc.append(roc_auc_score(y_test, rlc.predict(X_test)))
    probs_rlc = rlc.predict_proba(X_test)

# K plus proche voisins -----------------------------------------------------------------------

  with col2:
    expander = st.expander("Modèle KNN")

    knn = neighbors.KNeighborsClassifier(n_neighbors=39)
    knn.fit(X_train, y_train)
      
    expander.metric("Score train", "{:.2%}".format(knn.score(X_train, y_train)))
    expander.metric("Score test", "{:.2%}".format(knn.score(X_test, y_test)))
    expander.metric("Precision Score", "{:.2%}".format(precision_score(y_test, knn.predict(X_test))))

    y_pred = knn.predict(X_test)
    expander.write("Matrice de confusion :")
    expander.write(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

    # Sauvegarde des résultats
    models.append("K plus proches voisins")
    scores.append(knn.score(X_test, y_test))
    precision.append(precision_score(y_test, knn.predict(X_test)))
    rappel.append(recall_score(y_test, knn.predict(X_test)))
    roc.append(roc_auc_score(y_test, knn.predict(X_test)))
    probs_knn = knn.predict_proba(X_test)

# Arbre de décision -----------------------------------------------------------------------

  with col3:
    expander = st.expander("Modèle DTC")

    dtc = tree.DecisionTreeClassifier(max_depth=9)
    dtc.fit(X_train, y_train)  
        
    expander.metric("Score train", "{:.2%}".format(dtc.score(X_train, y_train)))
    expander.metric("Score test", "{:.2%}".format(dtc.score(X_test, y_test)))
    expander.metric("Precision Score", "{:.2%}".format(precision_score(y_test, dtc.predict(X_test))))

    y_pred = dtc.predict(X_test)
    expander.write("Matrice de confusion :")
    expander.write(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

    # Sauvegarde des résultats
    models.append("Decision Tree")
    scores.append(dtc.score(X_test, y_test))
    precision.append(precision_score(y_test, dtc.predict(X_test)))
    rappel.append(recall_score(y_test, dtc.predict(X_test)))
    roc.append(roc_auc_score(y_test, dtc.predict(X_test)))
    probs_dtc = dtc.predict_proba(X_test)

# Random Forest -----------------------------------------------------------------------

  with col4:
    expander = st.expander("Modèle RFC")

    rfc = ensemble.RandomForestClassifier(n_jobs=1) 
    rfc.fit(X_train, y_train)
    
    expander.metric("Score train", "{:.2%}".format(rfc.score(X_train, y_train)))
    expander.metric("Score test", "{:.2%}".format(rfc.score(X_test, y_test)))
    expander.metric("Precision Score", "{:.2%}".format(precision_score(y_test, rfc.predict(X_test))))

    y_pred = rfc.predict(X_test)
    expander.write("Matrice de confusion :")
    expander.write(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

    # Sauvegarde des résultats
    models.append("Random Forest")
    scores.append(rfc.score(X_test, y_test))
    precision.append(precision_score(y_test, rfc.predict(X_test)))
    rappel.append(recall_score(y_test, rfc.predict(X_test)))
    roc.append(roc_auc_score(y_test, rfc.predict(X_test)))
    probs_rfc = rfc.predict_proba(X_test)


# Comparaison des résultats -----------------------------------------------------------------------

  expander = st.expander("Comparaison des 4 modèles")

  # Recap des scores
  compare = pd.DataFrame(models)
  compare.columns = ['model']
  compare["accuracy"]=scores
  compare["precision"]=precision
  compare["rappel"]=rappel
  compare["roc"]=roc

  #Graphique de comparaison des résultats
  fig = plt.figure(figsize=(20,10))

#  compare.plot.bar(x = 'model', y=['accuracy', 'precision', 'rappel','roc'],stacked=False, rot=90)

  plt.bar(x = 'model', y=['accuracy', 'precision', 'rappel','roc'])

  plt.ylim([0.5, 1])
  plt.axhline(y=0.80, color='k', linewidth=2, linestyle='--')
  plt.title("Compare Models")
  expander.pyplot(fig)


  # Comparaison avec l'indice des ROC
  fig = plt.figure(figsize=(20,10))

  # Regression logistique
  fpr, tpr, seuils = roc_curve(y_test, probs_rlc[:,1])
  roc_auc = auc(fpr, tpr)
  plt.plot(fpr, tpr, color='green', lw=2, label='Modèle RLC (auc = %0.2f)' % roc_auc)

  # K plus proches voisins
  fpr, tpr, seuils = roc_curve(y_test, probs_knn[:,1])
  roc_auc = auc(fpr, tpr)
  plt.plot(fpr, tpr, color='blue', lw=2, label='Modèle KNN (auc = %0.2f)' % roc_auc)

  # Decision Tree
  fpr, tpr, seuils = roc_curve(y_test, probs_dtc[:,1])
  roc_auc = auc(fpr, tpr)
  plt.plot(fpr, tpr, color='orange', lw=2, label='Modèle DTC (auc = %0.2f)' % roc_auc)

  # Random Forest
  fpr, tpr, seuils = roc_curve(y_test, probs_rfc[:,1])
  roc_auc = auc(fpr, tpr)
  plt.plot(fpr, tpr, color='red', lw=2, label='Modèle RFC (auc = %0.2f)' % roc_auc)

  plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--', label='Aléatoire (auc = 0.5)')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('Taux faux positifs')
  plt.ylabel('Taux vrais positifs')
  plt.title('Courbe ROC pour modèle Random Forest')
  plt.legend(loc="lower right")
  expander.pyplot(fig)

  expander = st.expander("Choix du modèle")
  expander.write("Le modèle Random Forest semble le plus équilibré. Il permet de maximiser les positifs.")


### Page 4 - Pour aller plus loin

In [ ]:
%%writefile streamlit_app.py 

# ______________________________________________________________________________________________________
# 5/ BONUS
# ______________________________________________________________________________________________________


if page==pages[4]: 

  st.title("Estimation du nombre de leads sur la future campagne")

  from datetime import datetime
  #d = st.date_input("A quelle date souhaitez-vous lancer la campagne",datetime.date(2022, 7, 1))
  d = st.date_input("When's your birthday", datetime.date(2020, 8, 11))
  t = st.time_input("A combien estimez-vous le temps d'un appel client", datetime.time(8, 45))

  start_time = st.slider("When do you start?",value=d,format="MM/DD/YY")

# Exécution

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '16'
os.environ['NUMEXPR_NUM_THREADS'] = '8'
import numexpr as ne

# run streamlit
!streamlit run streamlit_app.py & public_url

In [ ]:
!streamlit run streamlit_app.py